In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

### Suppress most warnings for now

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Get SST Data

In [3]:
fTEMP = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ORAS5/1x1_grid/votemper/opa0_ext'
dTEMP = xr.open_dataset(fTEMP,decode_times=True)

### Make anomalies and Nino3.4

In [4]:
nino_total = dTEMP.votemper[:,0,:,:].sel(lat=slice(-5,5),
                                       lon=slice(210,260)).mean(dim='lat').mean(dim='lon')

nino_climo = nino_total.groupby('time.month').mean(dim='time')
nino_anom = nino_total.groupby('time.month') - nino_climo

sst = dTEMP.votemper[:,0,:,:]
sst_climo = sst.groupby('time.month').mean(dim='time')
sst_anom = sst.groupby('time.month') - sst_climo

X = sst_anom[0:600,:,:]

### Turn into categorization problem with 1's and 0's for warm and cold events

In [5]:
yy = nino_anom[0:600]
l_warm = ( yy > 0.5 )
l_cold = ( yy < 0.5 )
y = [1*l_warm,1*l_cold]
zz = np.transpose(np.asarray(y))

### Make a keras model

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from keras import initializers

Using TensorFlow backend.


In [7]:
inputdim = 180*360
model = Sequential()
initializers.RandomUniform() 

model.add(Dense(8, input_dim=inputdim,activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Fit the model to the data

In [ ]:
foo = np.asarray(X)
XX = foo.reshape(600,inputdim)
XX[np.isnan(XX)] = 0.0

model.fit(XX, zz, epochs=100, batch_size=60)


Epoch 1/100
600/600 [==============================] - 0s 725us/step - loss: 0.4035 - acc: 0.8217
Epoch 2/100
600/600 [==============================] - 0s 246us/step - loss: 0.1565 - acc: 0.9350
Epoch 3/100
600/600 [==============================] - 0s 253us/step - loss: 0.0940 - acc: 0.9550
Epoch 4/100
600/600 [==============================] - 0s 241us/step - loss: 0.0787 - acc: 0.9650
Epoch 5/100
600/600 [==============================] - 0s 236us/step - loss: 0.0400 - acc: 0.9850
Epoch 6/100
600/600 [==============================] - 0s 234us/step - loss: 0.0279 - acc: 0.9867
Epoch 7/100
600/600 [==============================] - 0s 239us/step - loss: 0.0229 - acc: 0.9883
Epoch 8/100
600/600 [==============================] - 0s 233us/step - loss: 0.0195 - acc: 0.9917
Epoch 9/100
600/600 [==============================] - 0s 235us/step - loss: 0.0184 - acc: 0.9933
Epoch 10/100
600/600 [==============================] - 0s 236us/step - loss: 0.0197 - acc: 0.9950
Epoch 11/100
600/6

### Apply Innvestigate

In [ ]:
import innvestigate
import innvestigate.utils

In [ ]:
# Strip softmax layer
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("deep_taylor", model)

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(XX)

In [ ]:
a.shape

### Reshape to time,lat,lon and plot

In [ ]:
a=a.reshape((600,180,360))
plt.pcolormesh(a[0,:,:])
plt.colorbar()